In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


In [20]:
chars = ""
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))
        
vocab_size = len(chars)

In [5]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([72,  0,  0,  0,  0,  0, 34, 48, 45,  1, 37, 55, 54, 44, 45, 58, 46, 61,
        52,  1, 37, 49, 66, 41, 58, 44,  1, 55, 46,  1, 29, 66,  0,  0, 42, 65,
         1, 26,  8,  1, 20, 58, 41, 54, 51,  1, 16, 41, 61, 53,  0,  0,  0, 34,
        48, 49, 59,  1, 42, 55, 55, 51,  1, 49, 59,  1, 44, 45, 44, 49, 43, 41,
        60, 45, 44,  1, 60, 55,  1, 53, 65,  1, 47, 55, 55, 44,  1, 46, 58, 49,
        45, 54, 44,  1,  3,  1, 43, 55, 53, 58])


In [10]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[61, 60,  1, 59, 48, 45,  1, 63],
        [45,  1, 20, 55, 58, 45, 59, 60],
        [55, 54, 60, 49, 54, 61, 45, 44],
        [54, 47,  1, 55, 61, 60,  1, 60]], device='cuda:0')
targets:
tensor([[60,  1, 59, 48, 45,  1, 63, 41],
        [ 1, 20, 55, 58, 45, 59, 60,  0],
        [54, 60, 49, 54, 61, 45, 44,  0],
        [47,  1, 55, 61, 60,  1, 60, 48]], device='cuda:0')


In [7]:

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context,'target is', target)

when input is tensor([72]) target is tensor(0)
when input is tensor([72,  0]) target is tensor(0)
when input is tensor([72,  0,  0]) target is tensor(0)
when input is tensor([72,  0,  0,  0]) target is tensor(0)
when input is tensor([72,  0,  0,  0,  0]) target is tensor(0)
when input is tensor([72,  0,  0,  0,  0,  0]) target is tensor(34)
when input is tensor([72,  0,  0,  0,  0,  0, 34]) target is tensor(48)
when input is tensor([72,  0,  0,  0,  0,  0, 34, 48]) target is tensor(45)


In [21]:
#embeding table
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
   
    def forward(self, index, targets):
        logits = self.token_embedding_table(index)
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = target.view(B*T)
        loss = F.cross_entropt(logits, targets)

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

TypeError: BigramLanguageModel.forward() missing 1 required positional argument: 'targets'